In [1]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os, cv2, math, statistics, datetime, itertools
import pandas as pd
import pylab as pl
import scipy.cluster.hierarchy as sch
import xarray as xr
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.express as px

#from datetime import datetime
from mpl_toolkits.basemap import Basemap
from PIL import Image
from matplotlib import cm
from pvlib import location
from pvlib.irradiance import get_extra_radiation
from pvlib.location import Location
from plotly import subplots
from plotly.subplots import make_subplots

In [2]:
# List containing the location name, Latitude/Longitude/Altitude of the surface stations and number which will be displayed on map
locations = [['brb', -15.60083, -47.71306, 1023, 'Brasilia'],
             ['cpa', -22.6896, -45.0062, 574, 'Cachoeira Paulista'],
             ['ptr', -9.0689, -40.3197, 387, 'Petrolina'],
             ['sms', -29.4428, -53.8231, 489, 'Sao Martinho da Serra']]

# Setting first time step t_0, this is the time which will be used to kickstart the forecast
t = "1511"

In [3]:
# Setting the source files path, this is where all files with the .nc extension are located
path = r"C:\Users\Usuario\Documents\Unifesp\Mestrado\arquivos\inteiros\01"
os.chdir(path)

# Finding the time difference between two consecutive frames
files = []
for file in os.listdir(path):
    files.append(file)

delta = pd.to_datetime(files[1][-10:-6], format='%H%M').minute - pd.to_datetime(files[0][-10:-6], format='%H%M').minute
del files

# Setting previous time step, two consecutive frames are necessary for the use of optical flow
t_0 = pd.to_datetime(t, format='%H%M') - datetime.timedelta(minutes=delta)
t_0 = str(t_0.hour) + str(t_0.minute)

# Creating list to store every netcdf file in which optical flow will be applied
goes_frames_of = []

# Looping thorugh goes files, adding to the list
for file in os.listdir(path):
    if file[-10:-6] == t_0 or file[-10:-6] == t:
        goes_frames_of.append(file)

In [4]:
# Looping through locations
for location in locations:
    
    # Looping through every .nc file in the directory
    for file in goes_frames_of:
        
        # Opening the dataset with xarray library
        ncfile = file
        ds = xr.open_dataset(ncfile)
        
        # Extracting the date from the file
        date = str(ds.attrs['date_created'])
    
        # Selecting the variables
        lon = ds['lon'].values
        lat = ds['lat'].values

        # Setting the value of k, this is the size of our box around the location
        k = 2.0
        
        # Setting the latitude and latitude
        lat_station = location[1]
        lon_station = location[2]
        lat_list = list(lat)
        lon_list = list(lon)
        
        # Finding the closest point in the dataset that matches our location latitude and longitude
        lat_start = lat_list.index(lat_list[min(range(len(lat_list)), key = lambda i: abs(lat_list[i]-(lat_station - k)))])
        lat_end = lat_list.index(lat_list[min(range(len(lat_list)), key = lambda i: abs(lat_list[i]-(lat_station + k)))])
        lon_start = lon_list.index(lon_list[min(range(len(lon_list)), key = lambda i: abs(lon_list[i]-(lon_station - k)))])
        lon_end = lon_list.index(lon_list[min(range(len(lon_list)), key = lambda i: abs(lon_list[i]-(lon_station + k)))])
        
        # Setting the new .nc file with our specified dimensions
        data = ds['CMI'][lat_start:lat_end, lon_start:lon_end]
        
        # Naming our file with the date and time
        file = location[0] + '_' + date[0:4] + '_' + date[5:7] + '_' + date[8:10] + '_' + date[11:13] + date[14:16]
        
        # Saving the cropped NetCDF file
        data.to_netcdf(path=fr"C:\Users\Usuario\Documents\Unifesp\Mestrado\arquivos\recortados\{file}.nc")
        ds.close()

In [5]:
cci = [['brb', [0.1054, 0.6152], [0.1990, 0.5511], [0.1206, 0.3708], [0.2270, 0.6498]],
      ['cpa', [0.1098, 0.4775], [0.1197, 0.5825], [0.0841, 0.4041], [0.1873, 0.7200]],
      ['ptr', [0.2035, 0.5848], [0.1337, 0.5902], [0.2171, 0.5740], [0.1883, 0.4949]],
      ['sms', [0.1689, 0.7349], [0.2305, 0.6181], [0.1314, 0.4914], [0.1616, 0.6251]]]

In [6]:
# Path where files will be saved
save_cluster_path = r'C:\Users\Usuario\Documents\Unifesp\Mestrado\arquivos\cluster'
ds_path = r'C:\Users\Usuario\Documents\Unifesp\Mestrado\arquivos\recortados'

for values in cci:
    location = values[0]
    clear_index = values[1][0]
    cloud_index = values[1][1]
    print(location, clear_index, cloud_index)
    
    for file in os.listdir(ds_path):
        if location in file:
            ds_2 = xr.open_dataset(ds_path + '\\' + file)
            file_name = file[0:-3] + '.png'
            ds_cmi = ds_2.CMI.values

            # Calculating CCI
            c_eff = (ds_cmi - clear_index)/(cloud_index - clear_index)

            # Replacing values 
            c_eff = np.where(c_eff < 0, 0, c_eff)
            c_eff = np.where(c_eff > 1, 1, c_eff)
            ds_2.close()

            # Rotating the array so it matches our geographical location, converting the file to 8-bit grayscale image
            im = Image.fromarray(np.uint8(cm.gist_earth(np.rot90(np.flip(c_eff.T), k=3))*255)).convert('L')
            im.save(save_cluster_path + '\\' + file_name)

brb 0.1054 0.6152
cpa 0.1098 0.4775
ptr 0.2035 0.5848
sms 0.1689 0.7349


In [7]:
path = r'C:\Users\Usuario\Documents\Unifesp\Mestrado\arquivos\cluster'
os.chdir(r'C:\Users\Usuario\Documents\Unifesp\Mestrado\arquivos\cluster')
persistence = []


for location in locations:
    values_persist = []
    for image in os.listdir(path):
        if t == image[-8:-4] and location[0] == image[0:3]:
            for i in range(8):
                values_persist.append(np.mean(mpimg.imread(image)[224:227, 224:227]))
    persistence.append(values_persist)

In [8]:
frame_folder = rf"C:\Users\Usuario\Documents\Unifesp\Mestrado\arquivos\cluster"
directory = "xx_xx"

for location in locations:
    
    if location[0] == 'brb':
        levels = 5
        winsize = 25
        iterations = 3
        
    elif location[0] == 'cpa':
        levels = 5
        winsize = 35
        iterations = 7 
        
    elif location[0] == 'ptr':
        levels = 5
        winsize = 25
        iterations = 5 
        
    else:
        levels = 7
        winsize = 35
        iterations = 7 
        
    files = []
    
    for file in os.listdir(rf"C:\Users\Usuario\Documents\Unifesp\Mestrado\arquivos\cluster"):
        if location[0] in file:
            files.append(file)
            
    for i in range(len(files)-1):

        if directory != files[i][9:14]:
            directory = files[i][9:14]
            parent_dir = rf"C:\Users\Usuario\Documents\Unifesp\Mestrado\arquivos\previstos\{location[0]}"
            output_folder = os.path.join(parent_dir, directory)

            try:
                os.mkdir(output_folder, 0o666)
            except FileExistsError:
                pass

            # Sequential images to be used for the optical flow
            frames = [files[i], files[i+1]]

            # Forecast window (how many future frames will be generated)
            forecast_window = int((120/delta) - 1)

            # Algorithm split into 2 loops, the first one goes through all available frames
            # The second loop creates forecast images from the previous generated forecast

            # Loop 1: available frames
            for i in range (1,len(frames)):

                # Setting file path
                nome_frame1 = os.path.join(frame_folder, frames[i-1])
                nome_frame2 = os.path.join(frame_folder,frames[i])
                #nome_saida = os.path.join(output_folder, 'FLOWMAP_FRAME{0}-{1}.png'.format(int(i-1), int(i)))
                nome_saida2 = os.path.join(output_folder, 'FORECAST_FRAME{0}.png'.format(int(i+1)))

                # Reading the images
                frame1 = cv2.imread(nome_frame1)
                prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
                frame2 = cv2.imread(nome_frame2)
                nxt = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY) 

                # Preparing the flowmap array
                hsv_mask = np.zeros_like(frame1) 
                hsv_mask[..., 1] = 255

                # Applying Farneback Optical Flow
                flow = cv2.calcOpticalFlowFarneback(prvs, nxt, None, 0.5, levels, winsize, iterations, 5, 1.2, 0) 
#mudar 31 pra 5-10 e 25 pra 15 e 30 e de 3 pra 5,7,9

                # Obtaining the magnitude and angle of the displacement vector
                mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1], angleInDegrees = True) 

                # Creating mask to be used on future frames
                mag_i = mag.astype(int)
                mag_i = np.where(mag_i == 0, 1, 0)

                # Splitting the vectors u and v from the optical flow
                v = flow[...,1]
                u = flow[...,0]

                ub = u
                vb = v

                # Saving the flowmap in 4 directions
                down_mask = np.where((ang > 45) & (ang <=135), 90, 0)
                up_mask = np.where((ang > 225) & (ang <=315), 270, 0)
                left_mask = np.where((ang > 135) & (ang <=225), 180, 0)
                right_mask = np.where((ang > 315) | (ang <=45), 0, 0)
                fin_mask = down_mask + up_mask + left_mask + right_mask

                # Saturating the top 1% of the magnitude to reduce the range of the values
                cutoff = np.percentile(mag, 99)
                mag[mag>cutoff] = cutoff

                # Filling the flowmap array
                hsv_mask[..., 0] = fin_mask    
                hsv_mask[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX) 

                # Converting the flowmap array to RGB and saving
                rgb_representation = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR) 
                #cv2.imwrite(nome_saida, rgb_representation)


                # Preparing the forecast frame. The multiplication by the magnitude mask allows the filling of the stationary areas,
                # this avoids the overwriting of pixels
                next_frame = frame2[:,:,0] * mag_i

                # Conducting the forecast of the next frame
                # It is necessary to go through each pixel, applying the dispplacement vector
                # Pixels moving outside of the frame and stationary pixels are ignored
                for i in range(frame2.shape[0]):
                    for j in range(frame2.shape[1]):

                        if (int(ub[i,j]==0) and int(vb[i,j]==0) ):
                            continue

                        if ((i + int(vb[i,j])) >= next_frame.shape[0]) or ((j + int(ub[i,j])) >= next_frame.shape[1]):
                            continue

                        next_frame[i + int(vb[i,j]), j + int(ub[i,j])] = frame2[i,j,0]


                # Applying a selective median filter, this removes deformations
                next_frame3 = cv2.medianBlur(next_frame.astype(np.float32), 5)    
                next_frame2 = np.where(next_frame == 0, next_frame3, next_frame)

                # Saving the forecasted frame and preparing the array for the next iteration
                cv2.imwrite(nome_saida2, next_frame2.astype(np.int64))
                next_frame = np.zeros( (next_frame2.shape[0],next_frame2.shape[1], 3) ).astype(int)
                next_frame[...,0] = next_frame2.astype(int)
                next_frame[...,1] = next_frame2.astype(int)
                next_frame[...,2] = next_frame2.astype(int)

            # Second loop: additional forecasts
            # The amount of iterations is equal to the forecast range
            for i in range(forecast_window):

                # Setting file path
                j = i + len(frames)
                #nome_saida = os.path.join(output_folder, 'FLOWMAP_FRAME{0}-{1}.png'.format(int(j-1), int(j)))
                nome_saida2 = os.path.join(output_folder, 'FORECAST_FRAME{0}.png'.format(int(j+1)))

                # # Reading the images
                frame1 = frame2.copy()
                frame2 = next_frame.copy()    
                prvs = frame1[...,0]
                nxt = frame2[...,0]

                # Preparing the flowmap array
                hsv_mask = np.zeros((frame1.shape)).astype(np.uint8)
                hsv_mask[..., 1] = 255

                # Applying Farnerback Optical Flow
                flow = cv2.calcOpticalFlowFarneback(prvs, nxt, None, 0.5, levels, winsize, iterations, 5, 1.2, 0) 

                # Obtaining the magnitude and angle of the displacement vector
                mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1], angleInDegrees = True) 

                # Creating mask to be used on future frames
                mag_i = mag.astype(int)
                mag_i = np.where(mag_i == 0, 1, 0)

                # Splitting the vectors u and v from the optical flow
                v = flow[...,1]
                u = flow[...,0]    

                ub = u
                vb = v

                # 4 Directions
                down_mask = np.where((ang > 45) & (ang <=135), 90, 0)
                up_mask = np.where((ang > 225) & (ang <=315), 270, 0)
                left_mask = np.where((ang > 135) & (ang <=225), 180, 0)
                right_mask = np.where((ang > 315) | (ang <=45), 0, 0)
                fin_mask = down_mask + up_mask + left_mask + right_mask

                # Saturating the top 1% of the magnitude to reduce the range of the values
                cutoff = np.percentile(mag, 99)
                mag[mag>cutoff] = cutoff

                # Filling the flowmap array
                hsv_mask[..., 0] = fin_mask    
                hsv_mask[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

                # Converting the flowmap array to RGB and saving
                rgb_representation = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR) 
                #cv2.imwrite(nome_saida, rgb_representation)

                # Preparing the forecast frame. The multiplication by the magnitude mask allows the filling of the stationary areas,
                # this avoids the overwriting of pixels
                next_frame = frame2[:,:,0] * mag_i

                # Conducting the forecast of the next frame
                # It is necessary to go through each pixel, applying the dispplacement vector
                # Pixels moving outside of the frame and stationary pixels are ignored
                for i in range(frame2.shape[0]):
                    for j in range(frame2.shape[1]):

                        if (int(ub[i,j]==0) and int(vb[i,j]==0) ):
                            continue

                        if ((i + int(vb[i,j])) >= next_frame.shape[0]) or ((j + int(ub[i,j])) >= next_frame.shape[1]):
                            continue

                        next_frame[i + int(vb[i,j]), j + int(ub[i,j])] = frame2[i,j,0]

                # Applying a selective median filter, this removes deformations
                next_frame3 = cv2.medianBlur(next_frame.astype(np.float32), 5)    
                next_frame2 = np.where(next_frame == 0, next_frame3, next_frame)

                # Saving the forecasted frame and preparing the array for the next iteration
                cv2.imwrite(nome_saida2, next_frame2.astype(np.int64))
                next_frame = np.zeros( (next_frame2.shape[0],next_frame2.shape[1], 3) ).astype(int)
                next_frame[...,0] = next_frame2.astype(int)
                next_frame[...,1] = next_frame2.astype(int)
                next_frame[...,2] = next_frame2.astype(int)

In [9]:
path = r'C:\Users\Usuario\Documents\Unifesp\Mestrado\arquivos\previstos\brb'
os.chdir(path)
timestamps = []

for folder in os.listdir(path):
    month = folder[0:2]
    day = folder[3:5]
    hour_start = str((pd.to_datetime(t, format='%H%M') + datetime.timedelta(minutes=delta)).hour) + ':' + str((pd.to_datetime(t, format='%H%M') + datetime.timedelta(minutes=delta)).minute)
    hour_end = str((pd.to_datetime(t, format='%H%M') + datetime.timedelta(hours=2)).hour) + ':' + str((pd.to_datetime(t, format='%H%M') + datetime.timedelta(hours=2)).minute)
    times = pd.date_range(start=f'2019-{month}-{day} {hour_start}', end=f'2019-{month}-{day} {hour_end}', freq=f'{delta}min')
    timestamps.append(times)
    
index = list(itertools.chain(*timestamps))

In [11]:
path = r'C:\Users\Usuario\Documents\Unifesp\Mestrado\arquivos\previstos'
os.chdir(path)
ceffs = []
location_name = []

for location in os.listdir(path):
    
    location_name.append(location)
    location_ceff = []
    new_path = path + rf'\{location}'
    
    for folder in os.listdir(new_path):
        new_path2 = new_path + rf'\{folder}'
        
        for forecast in os.listdir(new_path2):
            ceff_forecast = np.mean(mpimg.imread(new_path2 + rf'\{forecast}')[224:227, 224:227])
            location_ceff.append(ceff_forecast)
            
    ceffs.append(location_ceff)

data = dict(zip(location_name, ceffs))
df_forecast = pd.DataFrame(data, index=index)

In [12]:
extrarads = []

for day in df_forecast.index.day:
    date = datetime.datetime.strptime(f'01-{day}-2019', '%m-%d-%Y')
    extrarads.append(get_extra_radiation(date, method='nrel'))

location_data = []

for location in locations:
    data = Location(location[1], location[2], 'UTC', location[3], location[4])
    location_data.append(data)

ghi = location_data[0].get_clearsky(df_forecast.index).ghi/extrarads

for column in df_forecast.columns:
    df_forecast[column] = extrarads*((ghi - 0.05)*(1-df_forecast[column] + 0.05))

In [13]:
path = r'C:\Users\Usuario\Documents\Unifesp\Mestrado\arquivos\cluster'
os.chdir(r'C:\Users\Usuario\Documents\Unifesp\Mestrado\arquivos\cluster')
persistence = []


for location in locations:
    values_persist = []
    for image in os.listdir(path):
        if t == image[-8:-4] and location[0] == image[0:3]:
            for i in range(8):
                values_persist.append(np.mean(mpimg.imread(image)[224:227, 224:227]))
    persistence.append(values_persist)

test = {'brb_persist': persistence[0], 'cpa_persist': persistence[1], 
        'ptr_persist': persistence[2], 'sms_persist': persistence[3]}

df_persist = pd.DataFrame(test, index = index)
df_persist

extrarads = []

for day in df_persist.index.day:
    date = datetime.datetime.strptime(f'01-{day}-2019', '%m-%d-%Y')
    extrarads.append(get_extra_radiation(date, method='nrel'))

location_data = []

for location in locations:
    data = Location(location[1], location[2], 'UTC', location[3], location[4])
    location_data.append(data)

ghi = location_data[0].get_clearsky(df_persist.index).ghi/extrarads

for column in df_persist.columns:
    df_persist[column] = extrarads*((ghi - 0.05)*(1-df_persist[column] + 0.05))

df_persist = df_persist.groupby(df_persist.index.time).mean()
df_persist

,brb_persist,cpa_persist,ptr_persist,sms_persist
15:26:00,816.389045,695.830652,608.970060,459.333958
15:41:00,812.177581,692.370875,605.833500,456.958694
15:56:00,804.120345,685.629946,599.827976,452.419626
16:11:00,792.253561,675.638146,590.980488,445.737163
16:26:00,776.631048,662.440773,579.331158,436.941612
16:41:00,757.324023,646.097971,564.933092,426.073070
16:56:00,734.420939,626.684594,547.852254,413.181331
17:11:00,708.027363,604.290093,528.167372,398.325820


In [14]:
df_forecast

,brb,cpa,ptr,sms
2019-01-04 15:26:00,1085.216446,722.239170,1085.216446,889.316872
2019-01-04 15:41:00,1071.983437,786.270468,1079.148570,884.344348
2019-01-04 15:56:00,1067.979437,928.817515,1067.979437,875.191429
2019-01-04 16:11:00,939.589190,598.713997,1051.759355,861.899341
2019-01-04 16:26:00,1030.561944,588.357947,1030.561944,844.528414
...,...,...,...,...
2019-01-11 16:11:00,66.172064,1043.280024,614.862510,626.714223
2019-01-11 16:26:00,64.899456,1023.215870,603.037597,563.861163
2019-01-11 16:41:00,63.318204,998.285591,588.344807,457.298175
2019-01-11 16:56:00,61.435548,968.603315,570.851404,502.793487


In [15]:
df_forecast = df_forecast.groupby(df_forecast.index.time).mean()

## Dados para o Verão 

In [16]:
df = pd.read_csv(r'C:\Users\Usuario\Documents\Unifesp\Mestrado\dados sonda validacao\brb\2019\BRB_2019_001_a_031.csv', header=None, sep=';')
df[1] = df[1].values.astype('str')
df[2] = df[2].values.astype('str')

df_time = []

for minute in df[3]:
    df_time.append(str(datetime.timedelta(minutes=minute)))

df[3] = df_time

month = []
days = []

for day in df[2]:
    month.append(str(datetime.datetime.strptime(day, '%j').date().month))
    days.append(str(datetime.datetime.strptime(day, '%j').date().day))

df.index = pd.to_datetime(df[1].values + '-' + month + '-' + days + ' ' + df[3])
df.drop(columns=[0, 1, 2, 3], inplace=True)
df = df.groupby(df.index.time).mean()
df = df.rolling(10).mean()

mask1 = df.index >= df_forecast.index[0]
mask2 = df.index <= df_forecast.index[-1]

df = df.loc[mask1 & mask2]
df_brb = df.iloc[::15, :]

In [17]:
df = pd.read_csv(r'C:\Users\Usuario\Documents\Unifesp\Mestrado\dados sonda validacao\cpa\2019\CPA_2019_001_a_031.dat', header=None)
df.index = pd.to_datetime(df[0])
df = df[6]
df = df.groupby(df.index.time).mean()
df = df.rolling(10).mean()

mask1 = df.index >= df_forecast.index[0]
mask2 = df.index <= df_forecast.index[-1]

df = df.loc[mask1 & mask2]
df_cpa = df.iloc[::15]

In [18]:
df = pd.read_csv(r'C:\Users\Usuario\Documents\Unifesp\Mestrado\dados sonda validacao\ptr\2019\PTR_2019_001_a_031.dat', header=None)
df = df.loc[df[0] == 111]
df[1] = pd.to_numeric(df[1], downcast='integer')
df[1] = df[1].values.astype('str')
df[2] = df[2].values.astype('str')

df_time = []

for minute in df[3]:
    df_time.append(str(datetime.timedelta(minutes=minute)))

df[3] = df_time

month = []
days = []

for day in df[2]:
    month.append(str(datetime.datetime.strptime(day, '%j').date().month))
    days.append(str(datetime.datetime.strptime(day, '%j').date().day))

df.index = pd.to_datetime(df[1].values + '-' + month + '-' + days + ' ' + df[3])
df.drop(columns=[0, 1, 2, 3], inplace=True)
df = df.groupby(df.index.time).mean()
df = df.rolling(10).mean()

mask1 = df.index >= df_forecast.index[0]
mask2 = df.index <= df_forecast.index[-1]

df = df.loc[mask1 & mask2]
df = df.iloc[::15, :]
df_ptr = df[4]

In [19]:
df = pd.read_csv(r'C:\Users\Usuario\Documents\Unifesp\Mestrado\dados sonda validacao\sms\2019\SMS_2019_001_a_031.dat', header=None)
df = df.loc[df[0] == 108]
df[1] = pd.to_numeric(df[1], downcast='integer')
df[1] = df[1].values.astype('str')
df[2] = df[2].values.astype('str')

df_time = []

for minute in df[3]:
    df_time.append(str(datetime.timedelta(minutes=minute)))

df[3] = df_time

month = []
days = []

for day in df[2]:
    month.append(str(datetime.datetime.strptime(day, '%j').date().month))
    days.append(str(datetime.datetime.strptime(day, '%j').date().day))

df.index = pd.to_datetime(df[1].values + '-' + month + '-' + days + ' ' + df[3])
df.drop(columns=[0, 1, 2, 3], inplace=True)
df = df.groupby(df.index.time).mean()
df = df.rolling(10).mean()

mask1 = df.index >= df_forecast.index[0]
mask2 = df.index <= df_forecast.index[-1]

df = df.loc[mask1 & mask2]
df = df.iloc[::15, :]
df_sms = df[4]

In [20]:
trace0 = go.Scatter(x=[i for i in range(1,9)], y=df_brb[4],
                    mode='lines+markers', name='Observado', line=dict(color="blue") )

trace1 = go.Scatter(x=[i for i in range(1,9)], y=df_forecast['brb'],
                    mode='lines+markers', name='Previsto', line=dict(color="red"))

trace2 = go.Scatter(x=[i for i in range(1,9)], y=df_persist['brb_persist'],
                    mode='lines+markers', name='Persistência', line=dict(color="gray"))

trace3 = go.Scatter(x=[i for i in range(1,9)], y=df_cpa,
                    mode='lines+markers', name='Observado',line=dict(color="blue"),showlegend=False)

trace4 = go.Scatter(x=[i for i in range(1,9)], y=df_forecast['cpa'],
                    mode='lines+markers', name='Previsto',line=dict(color="red"),showlegend=False)

trace5 = go.Scatter(x=[i for i in range(1,9)], y=df_persist['cpa_persist'],
                    mode='lines+markers', name='Persistência', line=dict(color="gray"),showlegend=False)

trace6 = go.Scatter(x=[i for i in range(1,9)], y=df_ptr,
                     mode='lines+markers', name='Observado',line=dict(color="blue"),showlegend=False)

trace7 = go.Scatter(x=[i for i in range(1,9)], y=df_forecast['ptr'],
                     mode='lines+markers', name='Previsto',line=dict(color="red"),showlegend=False)

trace8 = go.Scatter(x=[i for i in range(1,9)], y=df_persist['ptr_persist'],
                    mode='lines+markers', name='Persistência', line=dict(color="gray"),showlegend=False)

trace9 = go.Scatter(x=[i for i in range(1,9)], y=df_sms,
                     mode='lines+markers', name='Observado',line=dict(color="blue"),showlegend=False)

trace10 = go.Scatter(x=[i for i in range(1,9)], y=df_forecast['sms'],
                     mode='lines+markers', name='Previsto',line=dict(color="red"),showlegend=False)

trace11 = go.Scatter(x=[i for i in range(1,9)], y=df_persist['sms_persist'],
                    mode='lines+markers', name='Persistência', line=dict(color="gray"),showlegend=False)

fig = subplots.make_subplots(rows=2, cols=2,
                             subplot_titles=['brb', 'cpa', 'ptr', 'sms'],
                             x_title='Previsão',
                             y_title='Radiação Global W/m²',
                             shared_yaxes=False,
                             shared_xaxes=True,
                             vertical_spacing=0.1,
                             horizontal_spacing=0.01)

fig.append_trace(trace0,1,1)
fig.append_trace(trace1,1,1)
fig.append_trace(trace2,1,1)
fig.append_trace(trace3,1,2)
fig.append_trace(trace4,1,2)
fig.append_trace(trace5,1,2)
fig.append_trace(trace6,2,1)
fig.append_trace(trace7,2,1)
fig.append_trace(trace8,2,1)
fig.append_trace(trace9,2,2)
fig.append_trace(trace10,2,2)
fig.append_trace(trace11,2,2)

fig.update_layout(width=1200, height=600, hovermode="x", template='simple_white',
                  title={'text': 'Previsão de 120 minutos 15:11 UTC'})
pyo.plot(fig)

'temp-plot.html'

In [ ]:
#brb_nrmse = str((np.sqrt((sum((df_forecast['brb'] - df_brb[4])**2))/len(df_forecast['brb'])))/np.mean(df_forecast['brb']))
#cpa_nrmse = str((np.sqrt((sum((df_forecast['cpa'] - df_cpa)**2))/len(df_forecast['cpa'])))/np.mean(df_forecast['cpa']))
#ptr_nrmse = str((np.sqrt((sum((df_forecast['ptr'] - df_ptr)**2))/len(df_forecast['ptr'])))/np.mean(df_forecast['ptr']))
#sms_nrmse = str((np.sqrt((sum((df_forecast['sms'] - df_sms)**2))/len(df_forecast['sms'])))/np.mean(df_forecast['sms']))

In [ ]:
#brb_rmse = str((np.sqrt((sum((df_forecast['brb'] - df_brb[4])**2))/len(df_forecast['brb']))))
#cpa_rmse = str((np.sqrt((sum((df_forecast['cpa'] - df_cpa)**2))/len(df_forecast['cpa']))))
#ptr_rmse = str((np.sqrt((sum((df_forecast['ptr'] - df_ptr)**2))/len(df_forecast['ptr']))))
#sms_rmse = str((np.sqrt((sum((df_forecast['sms'] - df_sms)**2))/len(df_forecast['sms']))))

In [ ]:
#df_rmse = pd.read_csv(r'C:\Users\Usuario\Desktop\1511.csv', sep=';')

In [ ]:
#df_rmse['brb_RMSE'][run] = brb_rmse.replace('.',',')
#df_rmse['cpa_RMSE'][run] = cpa_rmse.replace('.',',')
#df_rmse['ptr_RMSE'][run] = ptr_rmse.replace('.',',')
#df_rmse['sms_RMSE'][run] = sms_rmse.replace('.',',')

#df_rmse['brb_NRMSE'][run] = brb_nrmse.replace('.',',')
#df_rmse['cpa_NRMSE'][run] = cpa_nrmse.replace('.',',')
#df_rmse['ptr_NRMSE'][run] = ptr_nrmse.replace('.',',')
#df_rmse['sms_NRMSE'][run] = sms_nrmse.replace('.',',')

#df_rmse

In [ ]:
#df_rmse.to_csv(r'C:\Users\Usuario\Desktop\1511.csv', sep=';',index=False)